# Build subset of SEEG recording containing relevant HFOs and some background activity


### Check WD (change if necessary) and file loading

In [206]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/subset" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/subset"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/subset


## Choose if we are using the ripple band, fast ripple band or both at the same time

In [207]:
from utils.input import MarkerType, band_to_file_name

# chosen band
chosen_band = MarkerType.BOTH     # RIPPLE, FAST_RIPPLE, or BOTH

file_suffix = f'{band_to_file_name(chosen_band)}_band'

In [208]:
import numpy as np
import math
from utils.io import preview_np_array

seeg_file_common = f"../filter/results/synthetic/filtered_seeg_ch90-119"
seeg_file_name = f"{seeg_file_common}_{file_suffix}.npy"
recorded_data = np.load(f"{PATH_TO_FILE}{seeg_file_name}")

print("Data shape: ", recorded_data.shape)
print("First time steps: ", recorded_data[:10])

Data shape:  (245760, 30)
First time steps:  [[ 4.09480909e-04  9.68536667e-06  6.00492433e-04  2.11817278e-04
   8.19093464e-04 ... -1.27973846e-03 -3.00495254e-03 -1.17003699e-03
  -1.51614964e-03 -1.31151607e-03]
 [ 3.39510592e-03 -5.08286241e-04  4.07428527e-03  1.53467370e-03
   7.11611752e-03 ... -1.24625148e-02 -2.99039788e-02 -1.15449010e-02
  -1.33184926e-02 -1.29171986e-02]
 [ 9.48105758e-03 -7.24987603e-03  6.98102915e-03  2.11405073e-03
   2.63920311e-02 ... -5.33704203e-02 -1.27752620e-01 -4.94543299e-02
  -4.89733561e-02 -5.45703060e-02]
 [-4.69822442e-05 -4.14342767e-02 -1.93337945e-02 -1.30258934e-02
   5.37848010e-02 ... -1.26322770e-01 -2.87773455e-01 -1.14816642e-01
  -9.17076477e-02 -1.22544852e-01]
 [-6.51057965e-02 -1.29196840e-01 -1.08943042e-01 -5.89199385e-02
   6.80996985e-02 ... -1.53200083e-01 -2.87968860e-01 -1.34197504e-01
  -6.98435863e-02 -1.29662123e-01]
 [-1.68834248e-01 -2.22752900e-01 -2.02191806e-01 -8.46475789e-02
   7.59963013e-02 ...  1.79497541e

## Define Global Parameters of the Experiment

In [209]:
sampling_rate = 2048    # 2048 Hz
input_duration = 120 * (10**3)    # 120000 ms or 120 seconds
num_samples = recorded_data.shape[0]    # 2048 * 120 = 245760
num_channels = recorded_data.shape[1]   # 960

x_step = 1/sampling_rate * (10**3)  # 0.48828125 ms

## Define the Channels of Interest that will be used to extract the signal

In [210]:
ch_start_idx = 0    # 90  # Start index of the region of interest
ch_end_idx = ch_start_idx + 30 # End index of the region of interest 

### Extract the SEEG channels in the defined range from the SEEG data

In [211]:
# Extract the seeg_channels in the range [ch_start_idx, ch_end_idx]
relevant_seeg = recorded_data[:, ch_start_idx:ch_end_idx]

preview_np_array(relevant_seeg, "SEEG Channels")

SEEG Channels Shape: (245760, 30).
Preview: [[ 4.09480909e-04  9.68536667e-06  6.00492433e-04  2.11817278e-04
   8.19093464e-04 ... -1.27973846e-03 -3.00495254e-03 -1.17003699e-03
  -1.51614964e-03 -1.31151607e-03]
 [ 3.39510592e-03 -5.08286241e-04  4.07428527e-03  1.53467370e-03
   7.11611752e-03 ... -1.24625148e-02 -2.99039788e-02 -1.15449010e-02
  -1.33184926e-02 -1.29171986e-02]
 [ 9.48105758e-03 -7.24987603e-03  6.98102915e-03  2.11405073e-03
   2.63920311e-02 ... -5.33704203e-02 -1.27752620e-01 -4.94543299e-02
  -4.89733561e-02 -5.45703060e-02]
 [-4.69822442e-05 -4.14342767e-02 -1.93337945e-02 -1.30258934e-02
   5.37848010e-02 ... -1.26322770e-01 -2.87773455e-01 -1.14816642e-01
  -9.17076477e-02 -1.22544852e-01]
 [-6.51057965e-02 -1.29196840e-01 -1.08943042e-01 -5.89199385e-02
   6.80996985e-02 ... -1.53200083e-01 -2.87968860e-01 -1.34197504e-01
  -6.98435863e-02 -1.29662123e-01]
 ...
 [ 8.29896002e-01 -5.22494932e-01  1.10855384e+00 -3.81889266e-02
  -1.04171137e+00 ...  8.81004

## Import the Markers (Annotated Events) 
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [212]:
markers_seeg_file_name = f"{seeg_file_common}_markers.npy"
markers = np.load(f"{PATH_TO_FILE}{markers_seeg_file_name}")

print("Markers shape: ", markers.shape)
print("First time steps: ", markers[:10])

Markers shape:  (30, 42)
First time steps:  [[('Spike',   1000.  , 0.) ('Spike+Fast-Ripple',   4218.75, 0.)
  ('Ripple+Fast-Ripple',   6966.8 , 0.)
  ('Ripple+Fast-Ripple',   9793.95, 0.)
  ('Spike+Ripple+Fast-Ripple',  13189.  , 0.) ...
  ('Ripple+Fast-Ripple', 107859.  , 0.) ('Spike+Ripple', 111439.  , 0.)
  ('Ripple', 114551.  , 0.) ('Spike+Ripple', 116517.  , 0.)
  ('Spike+Ripple', 119000.  , 0.)]
 [('Spike+Ripple',   1000.  , 0.)
  ('Spike+Ripple+Fast-Ripple',   3917.97, 0.)
  ('Spike+Fast-Ripple',   6794.92, 0.) ('Fast-Ripple',   9094.73, 0.)
  ('Fast-Ripple',  12348.1 , 0.) ... ('Spike+Fast-Ripple', 107697.  , 0.)
  ('Fast-Ripple', 111079.  , 0.) ('Spike', 113382.  , 0.)
  ('Ripple+Fast-Ripple', 116656.  , 0.) ('Spike+Ripple', 119000.  , 0.)]
 [('Fast-Ripple',   1000.  , 0.) ('Spike',   3655.76, 0.)
  ('Spike',   7188.96, 0.) ('Spike',   9880.37, 0.)
  ('Spike+Ripple',  13890.6 , 0.) ...
  ('Spike+Fast-Ripple', 107924.  , 0.)
  ('Spike+Ripple+Fast-Ripple', 110238.  , 0.)
  ('Spi

### Get the markers with a certain SNR in a specific region
Let's use the markers with a SNR=15dB in the first region

In [213]:
relevant_markers = markers[ch_start_idx:ch_end_idx]
preview_np_array(relevant_markers, "relevant_markers")

relevant_markers Shape: (30, 42).
Preview: [[('Spike',   1000.  , 0.) ('Spike+Fast-Ripple',   4218.75, 0.)
  ('Ripple+Fast-Ripple',   6966.8 , 0.)
  ('Ripple+Fast-Ripple',   9793.95, 0.)
  ('Spike+Ripple+Fast-Ripple',  13189.  , 0.) ...
  ('Ripple+Fast-Ripple', 107859.  , 0.) ('Spike+Ripple', 111439.  , 0.)
  ('Ripple', 114551.  , 0.) ('Spike+Ripple', 116517.  , 0.)
  ('Spike+Ripple', 119000.  , 0.)]
 [('Spike+Ripple',   1000.  , 0.)
  ('Spike+Ripple+Fast-Ripple',   3917.97, 0.)
  ('Spike+Fast-Ripple',   6794.92, 0.) ('Fast-Ripple',   9094.73, 0.)
  ('Fast-Ripple',  12348.1 , 0.) ... ('Spike+Fast-Ripple', 107697.  , 0.)
  ('Fast-Ripple', 111079.  , 0.) ('Spike', 113382.  , 0.)
  ('Ripple+Fast-Ripple', 116656.  , 0.) ('Spike+Ripple', 119000.  , 0.)]
 [('Fast-Ripple',   1000.  , 0.) ('Spike',   3655.76, 0.)
  ('Spike',   7188.96, 0.) ('Spike',   9880.37, 0.)
  ('Spike+Ripple',  13890.6 , 0.) ...
  ('Spike+Fast-Ripple', 107924.  , 0.)
  ('Spike+Ripple+Fast-Ripple', 110238.  , 0.)
  ('Spik

### Build a subset of the SEEG recording joining data from the channels selected above that contain relevant HFOs

In [214]:
# Build the time_step values of the signal
time_vals = [val for val in np.arange(x_step, input_duration + x_step, x_step)]
print("Time values: ", time_vals[:10])

Time values:  [0.48828125, 0.9765625, 1.46484375, 1.953125, 2.44140625, 2.9296875, 3.41796875, 3.90625, 4.39453125, 4.8828125]


In [215]:
final_signal = np.empty(shape=(num_samples))    # Initialize the final signal
final_markers = []                  # List holding the final markers

# Variable holding the current marker idx for each channel
channels_curr_marker_idx = np.zeros(shape=(relevant_markers.shape[0]), dtype=int)
preview_np_array(channels_curr_marker_idx, "channels_curr_marker_idx")

channels_curr_marker_idx Shape: (30,).
Preview: [0 0 0 0 0 ... 0 0 0 0 0]


## Set the lenght of each segment to be extracted from a channel
Since we want to join the data from several channels in a single subset, we need to set the length of the segments to be extracted from each channel.

Furthermore, we also need to define a padding to avoid adding a HFO that occurs at the end of a segment since it may interfere with the next segment and be cut short.

In [216]:
segment_length = 500    # 500 / 1000 ms segments. Every 1000ms, there will be a relevant event (Ripple, Fast Ripple, or both)
# TODO: Could change to 1000ms for safety and increase the padding as well (to 300ms for example)

segment_end_padding = 200    # 200 / 300 ms padding at the end of each segment (Longest Ripples seem to last about 120ms according to Sofia?)

### Define the function that checks if the label of a marker belongs to the list of labels of interest

In [217]:
from utils.input import label_has_hfo_event, label_has_ripple, label_has_fast_ripple

# Function that dictates if a label is relevant
def is_relevant_label(label):
    if chosen_band == MarkerType.RIPPLE:
        return label_has_ripple(label)
    elif chosen_band == MarkerType.FAST_RIPPLE:
        return label_has_fast_ripple(label)
    else:
        return label_has_hfo_event(label)

In [218]:
# Randomly select a channel to start inserting annotated events
random_channel_offset = np.random.randint(0, relevant_markers.shape[0] - 1)
print("Random Channel Offset: ", random_channel_offset)

Random Channel Offset:  23


In [219]:
for curr_segment in range(0, input_duration, segment_length):
    time_start_idx = int(curr_segment / x_step)   # Start index of the segment in the time_vals array
    time_end_idx = int((curr_segment + segment_length) / x_step)   # End index of the segment in the time_vals array (exclusive)
    # print("time start idx: ", time_start_idx, "time end idx: ", time_end_idx)

    # Search for a relevant event in this segment
    starting_marker_idx = (int(curr_segment / segment_length) + random_channel_offset) % relevant_markers.shape[0]    # Get the marker index where we start searching for this segment
    # print("starting_marker_idx: ", starting_marker_idx)
    curr_marker_idx = starting_marker_idx

    segment_marker = None   # Variable to hold the marker of the current segment
    # Search for a relevant event in this time segment
    while True:
        # Check if the markers of the current channel have a relevant event inside the segment
        for inner_marker_idx in range(channels_curr_marker_idx[curr_marker_idx], relevant_markers.shape[1], 1):
            curr_marker = relevant_markers[curr_marker_idx][inner_marker_idx]

            # Check if the current marker is a relevant event (Ripple, Fast Ripple, or both)
            if is_relevant_label(curr_marker['label']):
                
                # print(curr_marker['position'] + segment_end_padding, curr_segment + segment_length)

                if curr_marker['position'] >= curr_segment and (curr_marker['position'] + segment_end_padding <= curr_segment + segment_length):
                    # print("curr_marker position: ", curr_marker['position'], "curr_segment: ", curr_segment, "segment_length: ", segment_length)
                    # Found a relevant event
                    segment_marker = curr_marker

                    # Update the current marker index for this channel to the next marker
                    channels_curr_marker_idx[curr_marker_idx] = inner_marker_idx + 1

                    break   # Stop searching for relevant events in this channel
            
            # Check if the current marker is outside the segment
            if curr_marker['position'] < curr_segment + segment_length:
                # Update the current marker index for this channel to the next marker of this channel
                channels_curr_marker_idx[curr_marker_idx] = inner_marker_idx + 1

        # If a relevant event is found, add the relevant event to the final signal
        if segment_marker is not None:
            # Add the relevant event's seeg data to the final signal
            final_signal[time_start_idx:time_end_idx] = relevant_seeg[time_start_idx:time_end_idx, curr_marker_idx]

            # Add the relevant event's marker to the final markers
            final_markers.append(segment_marker)  # TODO: Change this to the actual marker after being found
            break


        # Search in the markers of the next channel
        curr_marker_idx = (curr_marker_idx + 1) % relevant_markers.shape[0]    # Move to the next marker index

        # Check if we have reached the starting marker index
        if curr_marker_idx == starting_marker_idx:
            # We have searched all the markers in this segment (No relevant event found) 
            # -> Add current channel data to the final signal (with no relevant events)

            # Add background noise to the segment
            final_signal[time_start_idx:time_end_idx] = relevant_seeg[time_start_idx:time_end_idx, curr_marker_idx]
            break

### Preview the final signal and markers

In [220]:
preview_np_array(final_signal, "Final Signal")

Final Signal Shape: (245760,).
Preview: [ 0.00164447  0.01391946  0.04441551  0.04559582 -0.09086368 ...
  0.06471326 -0.56420797 -0.97813142 -1.05971408 -1.08183859]


In [221]:
final_markers = np.array(final_markers)
preview_np_array(final_markers, "Final Markers")

Final Markers Shape: (222,).
Preview: [('Ripple+Fast-Ripple',   1000.  , 0.)
 ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
 ('Spike+Ripple',   3521.  , 0.) ('Ripple+Fast-Ripple',   4134.77, 0.)
 ('Fast-Ripple',   4774.41, 0.) ...
 ('Spike+Ripple+Fast-Ripple', 115019.  , 0.)
 ('Spike+Fast-Ripple', 115777.  , 0.) ('Ripple', 116037.  , 0.)
 ('Ripple', 116721.  , 0.) ('Spike+Fast-Ripple', 119000.  , 0.)]


---

## Visualize the Subset of the SEEG Data

In [222]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or x_step?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

# Create the y arrays for the voltage plot representing the voltage of each electrode
v_yarrays = [final_signal]

## Create the Plot

In [223]:
# Create the plot
# List of tuples containing the y values and the legend label
hfo_y_arrays = [(final_signal, "Joined subset of SEEG Channels")]

# Create the SEEG Voltage plot
hfo_plot = create_fig(
    title=f"Subset sEEG Signal of Channels 90-119 ({band_to_file_name(chosen_band)} band)", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (μV)',
    x=x, 
    y_arrays=hfo_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    # y_range=Range1d(-0.05, 1.05)
)

# If there are more than 30 channels, hide the legend
if len(hfo_y_arrays) > 30:
    # Hide the legend
    hfo_plot.legend.visible = False

## Add Box Annotations to the plot to identify the marked HFOs (ground truth)

In [224]:
from bokeh.models import BoxAnnotation
# from utils.line_plot import color_map

show_markers = True    # Boolean to show the markers

color_map = {                  
    'Spike': 'red',
    'Fast-Ripple': 'blue',
    'Ripple': 'green',  
    'Spike+Ripple': 'yellow',
    'Spike+Fast-Ripple': 'pink',
    'Ripple+Fast-Ripple': 'cyan',
    'Spike+Ripple+Fast-Ripple': 'black'
}

confidence_range = 100          # TODO: Check this value. When the duration is missing (0), we consider the 200ms window around the marked position 
visited_markers = {}    # Avoid inserting multiple boxes for the same marker (only one of each label)
use_visited = False     # Boolean controlling if we remove duplicate markers
plot_instant = True     # Boolean to plot the markers as instant events or as boxes
instant_width = 100 # 20       # Width of the instant event for visualization purposes

if show_markers:
    for idx2, marker in enumerate(final_markers):
        # print("marker:", marker)
        
        if use_visited:
            # Check if the marker has already been visited and skip it if it has
            if marker['position'] in visited_markers:
                visited_labels = visited_markers[marker['position']]    # Get the labels that already have an annotation for this position
                if marker['label'] in visited_labels:
                    # print("Skipping marker", marker['position'], marker['label'])
                    continue    # Skip this marker
                else:
                    visited_labels.append(marker['label'])  # Add the label to the visited labels
            else:
                visited_markers[marker['position']] = [marker['label']] # Add the marker to the visited markers

        # Add a box annotation for each marker
        has_duration = marker['duration'] > 0
        
        confidence_constant = 0 if plot_instant or has_duration else confidence_range

        left = marker['position'] - confidence_constant
        right = marker['position'] + confidence_constant + instant_width
        box_color = color_map[marker['label']]  # Choose a color according to the label
        
        # if left < min_t or right > max_t:
        #     continue    # Skip this marker
        

        box = BoxAnnotation(left=left, right=right, fill_color=box_color, fill_alpha=0.35)
        # print("Added marker for channel: ", ch_idx, " at position: ", left)
        hfo_plot.add_layout(box)

## Show the Plot

In [225]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    bplt.show(hfo_plot)

## Export the plot to a file

In [226]:
export = True

if export:
    file_path = f"{PATH_TO_FILE}plots/seeg_filtered_subset_90-119_segment{segment_length}_{segment_end_padding}_{file_suffix}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="SEEG Data - Filtered Voltage dynamics across time")

    # Save the plot
    bplt.save(hfo_plot)

    # Close the plot
    bplt.curdoc().clear()
    bplt.reset_output()

## Write the final markers into a .npy file
Add an extra dimension (n_channels) to the markers array to match the shape of the other dataset

In [227]:
# To keep the same format as the original data, we will add an extra dimension to the final markers
final_markers = np.array([final_markers])
preview_np_array(final_markers, "Final Markers")

Final Markers Shape: (1, 222).
Preview: [[('Ripple+Fast-Ripple',   1000.  , 0.)
  ('Spike+Ripple+Fast-Ripple',   3206.54, 0.)
  ('Spike+Ripple',   3521.  , 0.) ('Ripple+Fast-Ripple',   4134.77, 0.)
  ('Fast-Ripple',   4774.41, 0.) ...
  ('Spike+Ripple+Fast-Ripple', 115019.  , 0.)
  ('Spike+Fast-Ripple', 115777.  , 0.) ('Ripple', 116037.  , 0.)
  ('Ripple', 116721.  , 0.) ('Spike+Fast-Ripple', 119000.  , 0.)]]


In [228]:
EXPORT_MARKERS = True
if EXPORT_MARKERS:
    # Define the parent folder to store the results
    parent_folder = f"results/seeg_filtered_subset_90-119_segment{segment_length}_{segment_end_padding}"
    # Create the folder to store the results if it does not exist
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)

    file_name = f"{parent_folder}/markers_{file_suffix}.npy"

    np.save(file_name, final_markers)   # Save the data to a numpy file (not stored in git due to size)

## Write the final subset of the SEEG data into a .npy file
Add an extra dimension (n_channels) to the signal array to match the shape of the other dataset

In [229]:
# To keep the same format as the original data, we will add an extra inner dimension to the final signal
final_signal = final_signal.reshape((num_samples, 1))
preview_np_array(final_signal, "Final Signal")

Final Signal Shape: (245760, 1).
Preview: [[ 0.00164447]
 [ 0.01391946]
 [ 0.04441551]
 [ 0.04559582]
 [-0.09086368]
 ...
 [ 0.06471326]
 [-0.56420797]
 [-0.97813142]
 [-1.05971408]
 [-1.08183859]]


In [230]:
EXPORT_SIGNAL = True
if EXPORT_SIGNAL:
    # Define the parent folder to store the results
    parent_folder = f"results/seeg_filtered_subset_90-119_segment{segment_length}_{segment_end_padding}"
    # Create the folder to store the results if it does not exist
    if not os.path.exists(parent_folder):
        os.makedirs(parent_folder)

    file_name = f"{parent_folder}/{file_suffix}.npy"
    np.save(file_name, final_signal)   # Save the data to a numpy file (not stored in git due to size)